In [46]:
import requests
from lxml import etree
import os
import time

if __name__ == '__main__':
    # 创建文件夹
    if not os.path.exists('./maogai/Round_2/'):
        os.mkdir('./maogai/Round_2/')

    session = requests.Session()
    detail_url = 'https://wlkc.ouc.edu.cn'
    # 使用现成的cookie直接绕过登录页面
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36',
        'Cookie': 'JSESSIONID=AC1DEAF6D6037A4B310E53885942338D; LOGIN=3139303230303331303330; UM_distinctid=17bcb2f4e579ab-0e328a4e032f8-a7d193d-144000-17bcb2f4e58315; SCREEN_NAME=4f566b646b73396b79642f396274486b665a503656773d3d; session_id=4025A5EAD4245D545DCB1E8FD2D1EE4E; s_session_id=864B98DCC97327775D6853902DCD0138; web_client_cache_guid=871e5dc2-27ae-44b8-898f-a44d97e217f2',
    }

    # 自动请求开始测试和提交 times_att为次数 每次请求间隔3s
    times_att = 10
    print('attempt started...')
    for i in range(0,times_att):
        # 请求测试
        attempt_url = 'https://wlkc.ouc.edu.cn/webapps/assessment/take/launch.jsp?course_assessment_id=_30131_1&course_id=_13492_1&new_attempt=1&content_id=_633969_1&step='
        atmp = session.get(url=attempt_url, headers=headers)
        
        # 拿到本次测试id
        do_url = 'https://wlkc.ouc.edu.cn/webapps/assessment/take/launch.jsp?course_assessment_id=_30131_1&course_id=_13492_1&content_id=_633969_1&step=null'
        do = session.get(url=do_url, headers=headers)
        tree_att = etree.HTML(do.text)
        atnum = tree_att.xpath('//div[@class="takeQuestionDiv "]/@id')[0]
        print('attemptid = ' + str(atnum))

        # 提交本次测试
        url2 = 'https://wlkc.ouc.edu.cn/webapps/assessment/do/take/saveAttempt?saveSequence=17&takePageId=1636771783088&course_assessment_id=_30131_1&course_id=_13492_1&content_id=_633969_1'
        data = {
            'blackboard.platform.security.NonceUtil.nonce.ajax': 'c87385ed-48d7-4d9c-8c8c-a9f6cb45826d',
            'data-submitted': 'true',
            'course_assessment_id': '_30131_1',
            'course_id': '_13492_1',
            'content_id': '_633969_1',
            'step': '',
            'original_user_id': '_47406_1',
            'save_and_submit': 'true',
            'timer_completion': '',
            'fileUploadType': '',
            'toggle_state': 'qShow',
            'current_question': '1',
            'current_attempt_item_id': atnum,
            'current_attempt_item_id_backup': atnum,
            'method': 'notajax',
            'saveonequestion': '',
        }
        fuck3 = session.post(url=url2, headers=headers, data=data)
        print('post finished! times: '+ str(i+1))
        time.sleep(3)
    print('attempt over!')
    print('--------------------')

    # 总尝试页面
    total_url = 'https://wlkc.ouc.edu.cn/webapps/gradebook/do/student/viewAttempts?method=list&course_id=_13492_1&outcome_definition_id=_95191_1&outcome_id=_2069239_1'
    page_text_tot = session.get(url=total_url, headers=headers)
    tree_tot = etree.HTML(page_text_tot.text)
    attempt_list = tree_tot.xpath('//div[@class=" columnStep clearfix"]//a/@href')

    # 为了去重使用字典保存键值对
    danxuan = {}
    duoxuan = {}
    panduan = {}
    cnt1 = 0
    cnt2 = 0
    cnt3 = 0

    fp1 = open('./maogai/Round_2/单选.txt', 'w', encoding='utf-8')
    fp2 = open('./maogai/Round_2/多选.txt', 'w', encoding='utf-8')
    fp3 = open('./maogai/Round_2/判断.txt', 'w', encoding='utf-8')

    times = 0
    print('download started...')
    for attempt in attempt_list:
        times = times + 1
        new_url = detail_url + attempt
        # print(new_url)
        page_text = session.get(url=new_url, headers=headers)
        tree = etree.HTML(page_text.text)
        pro_list = tree.xpath('//ul[@id="content_listContainer"]/li')

        cnt = 0
        # 每次尝试的内容进行爬取
        for li in pro_list:
            cnt = cnt + 1
            pro_name = li.xpath('normalize-space(.//div[@class="vtbegenerated inlineVtbegenerated"])')
            pro_answer_1 = li.xpath('.//div[@class="vtbegenerated inlineVtbegenerated"]//label/text()')
            pro_answer_1 = str(pro_answer_1)
            pro_answer_2 = li.xpath('normalize-space(.//span[@class="answerTextSpan"])')

            if cnt <= 5:
                if pro_name not in danxuan:
                    cnt1 = cnt1 + 1
                    danxuan[pro_name] = pro_answer_1
                    # fp1.write(str(cnt1) + '. ' + pro_name+'\n'+pro_answer_1+'\n')
            elif cnt <= 10:
                if pro_name not in duoxuan:
                    cnt2 = cnt2 + 1
                    duoxuan[pro_name] = pro_answer_1
                    # fp2.write(str(cnt2) + '. ' + pro_name + '\n' +pro_answer_1+'\n')
            else:
                if pro_name not in panduan:
                    cnt3 = cnt3 +1
                    panduan[pro_name] = pro_answer_2
                    # fp3.write(str(cnt3) + '. ' + pro_name + '\n'+pro_answer_2+'\n')
        print('times: ' + str(times) + ' 单选: ' + str(cnt1) + ',多选：' + str(cnt2) + ',判断: ' + str(cnt3) )

    # 将字典进行按key排序实现不同章节分类进行存储
    num = 0
    for pro in sorted(danxuan):
        num = num + 1
        fp1.write(str(num)+'. '+ pro + '\n' + danxuan[pro] + '\n')
    num = 0
    for pro in sorted(duoxuan):
        num = num + 1
        fp2.write(str(num)+'. ' + pro + '\n' + duoxuan[pro] + '\n')
    num = 0
    for pro in sorted(panduan):
        num = num + 1
        fp3.write(str(num)+'. ' + pro + '\n' + panduan[pro] + '\n')

    fp1.close()
    fp2.close()
    fp3.close()
    print('total: ' + str(cnt1+cnt2+cnt3) + ' problems have been downloaded.')


attempt started...
attemptid = _16646286_1
post finished! times: 1
attemptid = _16646303_1
post finished! times: 2
attemptid = _16646320_1
post finished! times: 3
attemptid = _16646337_1
post finished! times: 4
attemptid = _16646354_1
post finished! times: 5
attemptid = _16646371_1
post finished! times: 6
attemptid = _16646388_1
post finished! times: 7
attemptid = _16646405_1
post finished! times: 8
attemptid = _16646422_1
post finished! times: 9
attemptid = _16646439_1
post finished! times: 10
attempt over!
--------------------
download started...
times: 1 单选: 5,多选：5,判断: 5
times: 2 单选: 9,多选：9,判断: 10
times: 3 单选: 14,多选：13,判断: 13
times: 4 单选: 17,多选：18,判断: 16
times: 5 单选: 19,多选：19,判断: 20
times: 6 单选: 21,多选：21,判断: 23
times: 7 单选: 22,多选：22,判断: 26
times: 8 单选: 23,多选：24,判断: 27
times: 9 单选: 24,多选：25,判断: 28
times: 10 单选: 25,多选：26,判断: 30
times: 11 单选: 28,多选：26,判断: 31
times: 12 单选: 30,多选：28,判断: 31
times: 13 单选: 31,多选：30,判断: 31
times: 14 单选: 33,多选：31,判断: 31
times: 15 单选: 33,多选：33,判断: 31
times: 16